In [ ]:
# Let's keep our notebook clean, so it's a little more readable!
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
# %matplotlib notebook

In [ ]:
# Activate only if running from Google Collab
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/
#
# %ls

# Using machine learning to predict age from rs-fMRI

The goal is to extract data from several rs-fmri images, and use that data as features in a machine-learning model. We will integrate what we've learned in the previous machine-learning lecture to build an unbiased model and test it on a left out sample.

![feat_xtrct](https://ars.els-cdn.com/content/image/1-s2.0-S1053811919301594-gr1.jpg)

Link to slides: https://github.com/netneurolab/ipn-summer-school/blob/main/lectures/2021-06-29/13-15/20210629_ipn-summerschool_presentation_IntroML.pdf

# Preparing the data

In [ ]:
from nilearn import datasets
from glob import glob
import os

wdir = '/Users/estefanysuarez/nilearn_data/development_fmri'
data = datasets.fetch_development_fmri(n_subjects=2, reduce_confounds=True, data_dir=None, resume=True, verbose=1, age_group='both')

In order to do our machine learning, we will need to extract features from our rs-fmri images.

Specifically, we will extract signals from a brain parcellation and compute a correlation matrix, representing regional coactivation between regions.

We will practice on one subject first, then we'll extract data for all subjects.

## 1. Retrieve the atlas for extracting features and an example subject

Since we're using rs-fmri data, it makes sense to use an atlas defined using rs-fmri data

This paper has many excellent insights about what kind of atlas to use for an rs-fMRI machine learning task. See in particular Figure 5.

https://www.sciencedirect.com/science/article/pii/S1053811919301594?via%3Dihub

Let's use the MIST atlas, created here in Montreal using the BASC method. This atlas has multiple resolutions, for larger networks or finer-grained ROIs. Let's use a 64-ROI atlas to allow some detail, but to ultimately keep our connectivity matrices manageable.

Here is a link to the MIST paper: https://mniopenresearch.org/articles/1-3

In [ ]:
from nilearn import datasets

parcellations = datasets.fetch_atlas_basc_multiscale_2015(version='sym')
atlas_filename = parcellations.scale064

print('Atlas ROIs are located in nifti image (4D) at: %s' %
       atlas_filename)

Let's have a look at that atlas

In [ ]:
from nilearn import plotting

plotting.plot_roi(atlas_filename, draw_cross=False)

Great, let's load an example 4D fmri time-series for one subject

In [ ]:
fmri_filenames = data.func[0]
print(fmri_filenames)

Let's have a look at the image! Because it is a 4D image, we can only look at one slice at a time. Or, better yet, let's look at an average image!

In [ ]:
from nilearn import image 

averaged_Img = image.mean_img(image.mean_img(fmri_filenames))
plotting.plot_stat_map(averaged_Img)

In [ ]:
plotting.view_img(averaged_Img)

## 2. Extract signals on a parcellation defined by labels

To do so, we are going to use the NiftiLabelsMasker. 

So we've loaded our atlas and 4D data for a single subject. Let's practice extracting features!

In [ ]:
from nilearn.input_data import NiftiLabelsMasker

masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True, 
                           memory='nilearn_cache', verbose=1)

# Here we go from nifti files to the signal time series in a numpy
# array. Note how we give confounds to be regressed out during signal
# extraction
conf = data.confounds[0]
time_series = masker.fit_transform(fmri_filenames, confounds=conf)

In [ ]:
print(type(time_series))
print(time_series.shape)

What are these "confounds" and how are they used? 

In [ ]:
import pandas
conf_df = pandas.read_table(conf)
conf_df.head()

In [ ]:
print(conf_df.shape)

## 3. Compute and display a correlation matrix

In [ ]:
from nilearn.connectome import ConnectivityMeasure

correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([time_series])[0]
correlation_matrix.shape

Plot the correlation matrix

In [ ]:
import numpy as np
# Mask the main diagonal for visualization:
np.fill_diagonal(correlation_matrix, 0)

# The labels we have start with the background (0), hence we skip the
# first label
plotting.plot_matrix(correlation_matrix, figure=(10, 8), 
                     labels=range(time_series.shape[-1]),
                     vmax=0.8, vmin=-0.8, reorder=False)

# matrices are ordered for block-like representation

## 4. Extract features from the whole dataset

We can now use a for loop to iterate through each 4D image and use the same techniques we learned above to extract rs-fMRI connectivity features from every subject.

# Machine Learning Pipeline

## 1. Load the data

<p align="center">
<img src="Imgs/SampFeat.png" alt="terms" width="300"/>
</p>

### Load input features - X matrix

In [ ]:
import numpy as np

feat_file = 'MAIN_BASC064_subsamp_features.npz'
X_features = np.load(feat_file)['a']

n_samples, n_features = X_features.shape
print(f'Our data set contains {n_samples} samples and {n_features} features.')

Now that we've got our features, we can visualize them as a matrix!

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X_features, aspect='auto')
plt.colorbar()
plt.title('feature matrix')
plt.xlabel('features')
plt.ylabel('subjects')
plt.show()
plt.close()

### Load output labels (our target data) - Y vector

In [ ]:
import os
import pandas

# Let's load the phenotype data
pheno_path = os.path.join('participants.tsv')

pheno = pandas.read_csv(pheno_path, sep='\t').sort_values('participant_id')
pheno.head()

Looks like there is a column labeling age. Let's capture it in a variable!

In [ ]:
y_age = pheno['Age']

Maybe we should have a look at the distribution of our target variable!

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.displot(y_age)
plt.show()

## 2. Training/test data split

Here, we will divide our data set into *training* and *test* sets. We will use the training set to fit and tweak our model's parameters. The test set will be exclusively used to validate our model, and therefore it will be set aside and **WE WILL NOT TOUCH IT UNTIL THE VERY END!!!**. 

<p align="center">
<img src="Imgs/splitdata.png" alt="terms" width="300"/>
</p>

Now, we want to be sure that our training and test samples are matched! 
We can do that with a "stratified split". This dataset has a variable indicating AgeGroup. We can use that to make sure our training and testing sets are balanced!

In [ ]:
age_class = pheno['AgeGroup']
age_class.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the sample into training/test sets with a 60/40 ratio, and 
# stratify by age class, and also shuffle the data.

X_train, X_test, y_train, y_test = train_test_split(
                                                    X_features, # x
                                                    y_age, # y
                                                    test_size = 0.4, # 60%/40% split  
                                                    shuffle = True, # shuffle dataset
                                                                    # before splitting
                                                    stratify = age_class,  # keep
                                                                           # distribution
                                                                           # of ageclass
                                                                           # consistent
                                                                           # betw. train
                                                                           # & test sets.
                                                    random_state = 123 # same shuffle each
                                                                       # time
                                                                       )

# print the size of our training and test groups
print('Training sample:', len(X_train))
print('Test sample:', len(X_test))

Let's visualize the distributions to be sure they are matched

In [ ]:
sns.histplot(y_train, label='train', stat='density', kde=True, color=sns.color_palette()[0])
sns.histplot(y_test, label='test', stat='density', kde=True, color=sns.color_palette()[1])
plt.show()
plt.close()

## 3. Train your first ML model!

Machine-learning can get fancy pretty quickly. We'll start with a fairly standard regression model called a **Support Vector Regressor (SVR)**. 

While this may seem unambitious, simple models can be very robust. And we probably don't have enough data to create more complex models (but we can try later).

For more information, see this excellent resource:
https://hal.inria.fr/hal-01824205

Let's fit our first model!

<p align="center">
<img src="Imgs/modfit.png" alt="terms" width="400"/>
</p>

In [ ]:
from sklearn.svm import SVR

# define the model
l_svr = SVR(kernel='linear') 

# fit the model
l_svr.fit(X_train, y_train) 

## 4. Validate your model
Well... that was easy. Now let's see how well the model learned the data!

<p align="center">
<img src="Imgs/modval.png" alt="terms" width="800"/>
</p>

But, two questions first:

1. What happens if I validate my model with my **train** set? 
2. What happens if I validate my model with the **test** set?

Let's try with the **training** set:

In [ ]:
# predict the training data based on the model
y_pred = l_svr.predict(X_train) 

# caluclate the model accuracy
acc = l_svr.score(X_train, y_train) # the default score for the SVR model is the coefficient of determination R2

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_true=y_train,y_pred=y_pred)

Let's view our results and plot them all at once!

In [ ]:
# print results
print('Accuracy (R2) = ', acc) 
print('MAE           = ', mae) 

sns.regplot(y_pred,y_train)
plt.xlabel('Predicted Age')
# plt.show()

HOLY COW! Machine-learning is amazing!!! Almost a perfect fit :O ... which means there's must be something wrong. What's the problem here?

## 5. Cross-Validation (CV)

<p align="center">
<img src="Imgs/CV.png" alt="terms" width="300"/>
</p>

In [ ]:
from sklearn.model_selection import train_test_split

# Split the sample into training/validation sets with a 75/25 ratio, and 
# stratify by age class, and also shuffle the data.

age_class2 = pheno.loc[y_train.index,'AgeGroup']

X_train2, X_val, y_train2, y_val = train_test_split(
                                                    X_train, # x
                                                    y_train, # y
                                                    test_size = 0.25, # 75%/25% split  
                                                    shuffle = True, # shuffle dataset
                                                                    # before splitting
                                                    stratify = age_class2,  # keep
                                                                           # distribution
                                                                           # of ageclass
                                                                           # consistent
                                                                           # betw. train
                                                                           # & test sets.
                                                    random_state = 123 # same shuffle each
                                                                       # time
                                                                       )

# print the size of our new training and validation groups
print('(New) Training sample:', len(X_train2))
print('Validation sample:', len(X_val))

In [ ]:
from sklearn.metrics import mean_absolute_error

# fit model just to training data
l_svr.fit(X_train2,y_train2)

# predict the *test* data based on the model trained on X_train2
y_pred = l_svr.predict(X_val) 

# caluclate the model accuracy
acc = l_svr.score(X_val, y_val) 
mae = mean_absolute_error(y_true=y_val,y_pred=y_pred)

In [ ]:
# print results
print('CV accuracy (R2) = ', acc)
print('CV MAE           = ', mae)

sns.regplot(y_pred,y_val)
plt.xlabel('Predicted Age')
plt.show()

Not perfect, but as predicting with unseen data goes, not too bad! Especially with a training sample of "only" 69 subjects. But we can do better in terms of estimating the accuracy of our model's performance!

For example, we can "increase" the size of our training set by using 10-fold cross-validation (CV) instead. 

## 6. Using *k*-fold CV to accurately assess model performance

<p align="center">
<img src="Imgs/KCV1.png" alt="terms" width="900"/>
</p>


In [ ]:
from sklearn.model_selection import cross_val_predict, cross_val_score

# estimate CV scores
n_folds = 5
acc = cross_val_score(l_svr, X_train, y_train, cv=n_folds)
mae = cross_val_score(l_svr, X_train, y_train, cv=n_folds, scoring='neg_mean_absolute_error')
print(f"average ({n_folds}-fold) CV accuracy (R2) = %0.2f and standard deviation of %0.2f" % (acc.mean(), acc.std()))
print(f"average ({n_folds}-fold) CV MAE           = %0.2f and standard deviation of %0.2f" % (mae.mean(), mae.std()))

We can also look at the accuracy of the predictions for each fold of the cross-validation:

In [ ]:
for i in range(n_folds):
    print('Fold {} -- Acc = {}, MAE = {}'.format(i, acc[i], mae[i]))

Or we can also look at the overall accuracy of the model. In this case the data is split according to the CV parameter. Each sample belongs to exactly one validation set, and its prediction is computed with the model fitted on the corresponding training set.

In [ ]:
from sklearn.metrics import r2_score

y_pred = cross_val_predict(l_svr, X_train, y_train, cv=n_folds) # here we use our original training set
overall_acc = r2_score(y_train, y_pred)
overall_mae = mean_absolute_error(y_train,y_pred)

print('Overall CV R2:', overall_acc)
print('Overall CV MAE:', overall_mae)

sns.regplot(y_pred, y_train)
plt.xlabel('Predicted Age')
plt.show()

## 7. Tweak your model: *k*-fold CV for model selection

It's very important to learn when and where its appropriate to "tweak" your model.

Since we have done all of the previous analysis in our training data, it's fine to try out different models. But we **absolutely cannot** "test" it on our left out data. If we do, we are in great danger of overfitting.

It is not uncommon to try other models, or tweak hyperparameters. In this case, due to our relatively small sample size, we are probably not powered sufficiently to do so, and we would once again risk overfitting. However, for the sake of demonstration, we will do some tweaking. 

<p align="center">
<img src="Imgs/KCV2.png" alt="terms" width="400"/>
</p>



In [ ]:
from sklearn.svm import SVR

# define models and set hyperparameters
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_rbf = SVR(kernel='rbf', C=100, gamma='auto') #0.1, epsilon=.1)
svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=2) #, epsilon=.1, coef0=1)

models = {'svr_lin':svr_lin, 'svr_rbf':svr_rbf, 'svr_poly':svr_poly}

# perform k-fold CV with each model and evaluate performance
n_folds = 5
for name, model in models.items():
    print(f'\nPerformance for the {name.upper()} model:')
    acc = cross_val_score(model, X_train, y_train, cv=n_folds)
    mae = cross_val_score(model, X_train, y_train, cv=n_folds, scoring='neg_mean_absolute_error')
    print(f" average ({n_folds}-fold) CV accuracy (R2) = %0.2f with a standard deviation of %0.2f" % (acc.mean(), acc.std()))
    print(f" average ({n_folds}-fold) CV MAE           = %0.2f with a standard deviation of %0.2f" % (mae.mean(), mae.std()))
    

Thanks to our k-fold CV analyses, we can identify the *best* candidate model. Note however that the SVR model possesses a number of *hyperparameters* that could be tuned to potentially improve our predictions. The type of kernel is a hyperparameter itself. Let's try this! 

## 8. Tweak your model: *k*-fold CV + Grid Search for hyperparameter tuning

To tune our model hyperparameters we can combine grid searh and cross validation!

<p align="center">
<img src="Imgs/KCV3.png" alt="terms" width="400"/>
</p>

SVR model documentation: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# let's create our grid search 
tunning_params = [{'kernel': ['poly', 'linear', 'rbf'], 'gamma': [1e-3, 1e-4, 'scale', 'auto'],
                   'C': [1, 10, 100, 1000], 'degree':[2,3,4,5], 'epsilon':[.1], 'coef0':[1]}]

print("Tuning hyper-parameters ... ")
n_folds = 5
clf = GridSearchCV(estimator=SVR(), param_grid=tunning_params, cv=n_folds)#, scoring='neg_mean_absolute_error')
clf.fit(X_train, y_train)

print("\nGrid scores on development set:")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    
print(f"\nBest score (R2): {clf.best_score_}")
# print(f"\nBest score (MAE): {clf.best_score_}")
print("Best parameters set found on development set:")
opt_hyperparams = clf.best_params_
print(opt_hyperparams)

Thanks to our combined *k*-fold CV + Grid Search analysis, we have found the *best* model and the *best* set of model hyperparameters that maximize our (cross-validated) accuracy score, now what? 

## 9.  Ensemble methods and the *bias-variance* trade-off 

Now that we have selected the model and its hyper-parameters, we can now use ensemble methods, such as **bagging** and **boosting**, to improve the bias-variance trade-off of our model.  

In general, ensemble methods provide a way to reduce overfitting. **Bagging** improves the trade-off between bias and variance by slightly increasing the bias term, but allowing for a larger reduction of the variance term. Thefore, **bagging** methods are better suited for **strong and complex** models in which the variance is high.

On the other hand, **Boosting** improves the trade-off between bias and variance by slightly increasing the variance term, but allowing for a larger reduction of the bias term. Hence, **boosting** methods usually work best with **weak and simple** models in which the bias is high. 

These slight changes in the bias and/or the variance term *generally* result in a lower overall error and thus a better performance. 

<p align="center">
<img src="Imgs/ensemble.png" alt="terms" width="700"/>
</p>

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import cross_val_score

estimators = [("SVR", SVR(**opt_hyperparams)),
              ("Bagging(SVR)", BaggingRegressor(SVR(**opt_hyperparams))),
              ("Boosting(SVR)", AdaBoostRegressor(SVR(**opt_hyperparams)))
             ]

# Loop over estimators to compare
n_folds = 5
for n, (name, estimator) in enumerate(estimators):
    print(f'\nPerformance for the {name.upper()} model:')
    acc = cross_val_score(estimator, X_train, y_train, cv=n_folds)
    mae = cross_val_score(estimator, X_train, y_train, cv=n_folds, scoring='neg_mean_absolute_error')
    print(f" average ({n_folds}-fold) CV accuracy (R2) = %0.2f with a standard deviation of %0.2f" % (acc.mean(), acc.std()))
    print(f" average ({n_folds}-fold) CV MAE           = %0.2f with a standard deviation of %0.2f" % (mae.mean(), mae.std()))


### Bias-variance trade-off

In [ ]:
n_repeat = 4
n_val = 13

age_class2 = pheno.loc[y_train.index,'AgeGroup']
X_train2, X_val, y_train2, y_val = train_test_split(
                                                    X_train, 
                                                    y_train, 
                                                    test_size = n_val,  
                                                    shuffle = True, 
                                                    stratify = age_class2,
                                                    )

X_train2 = np.split(X_train2, n_repeat)
y_train2 = np.split(y_train2, n_repeat)

In [ ]:
estimators = [("SVR", SVR(**opt_hyperparams)),
              ("Bagging(SVR)", BaggingRegressor(SVR(**opt_hyperparams))),
              ("Boosting(SVR)", AdaBoostRegressor(SVR(**opt_hyperparams)))
             ]

n_estimators = len(estimators)

# Loop over estimators to compare
for n, (name, estimator) in enumerate(estimators):
    
    # Compute predictions
    y_predict = np.zeros((n_val, n_repeat))

    for i in range(n_repeat):
        estimator.fit(X_train2[i], y_train2[i])
        y_predict[:, i] = estimator.predict(X_val)

    # Bias^2 + Variance + Noise decomposition of the mean squared error
    y_error = np.zeros(n_val)

    for i in range(n_repeat):
        y_error += (y_val - y_predict[:, i]) ** 2

    y_error /= (n_repeat * n_repeat)
    y_bias = (y_val - np.mean(y_predict, axis=1)) ** 2
    y_var = np.var(y_predict, axis=1)
    
    print("{0}: {1:.4f} (error) = {2:.4f} (bias^2) "
          " + {3:.4f} (var)".format(name,
                                    np.mean(y_error),
                                    np.mean(y_bias),
                                    np.mean(y_var),))


Although neither ensemble method yields better performance compared to the single estimator, notice how **boosting** contributes to reduce the bias of the model, whereas **bagging** contributes to reduce its variance!!! 

Which method is then the best? It really depends on the application. Some applications require a higher accuracy (low bias), whereas other prefer to sacrifice a bit of accuracy while guaranteeing a higher precision (low variance). There is always a trade-off!!!

## 10.  Model deployment

Ok, so now that we have found the optimal model and its hyperparameters its time to release a working version of the model for "production". This process implies at least two things:

* **Report model performance**
* **Prepare model for production**  

Because we are not planning to make further changes in the model, we can now use our entire data set to report our model's performance, and to train our final model! 

### Reporting model performance

In [ ]:
estimator = SVR(**opt_hyperparams)
n_folds = 5
acc = cross_val_score(estimator, X_features, y_age, cv=n_folds)
mae = cross_val_score(estimator, X_features, y_age, cv=n_folds, scoring='neg_mean_absolute_error')

print("Model: SVR")
print(f"Model hyperparameters:{opt_hyperparams}")

print(f"\nModel performance:")
print(f" average ({n_folds}-fold) CV accuracy (R2) = %0.2f with a standard deviation of %0.2f" % (acc.mean(), acc.std()))
print(f" average ({n_folds}-fold) CV MAE           = %0.2f with a standard deviation of %0.2f" % (mae.mean(), mae.std()))

print(f"\nModel performance per fold:")
for i in range(n_folds):
    print(' Fold {} -- Acc = {}, MAE = {}'.format(i, acc[i], mae[i]))

### Prepare model for production

In [ ]:
estimator.fit(X_features,y_age)

## Final remarks

1. The MORE DATA the better!
2. BUT ... hopefully your data is as heterogeneous as possible to avoid BIASES in the model! 
3. ALWAYS get familiar with your data first --> use visualization tools! 
4. Do NOT touch your test data set until the very END!
5. Always go from simple to complex, especially if you do not have a lot observations!